# Milestone 1

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### Downloading the data 
rubric={correctness:10}

1. Download the data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) to your local computer using the [figshare API](https://docs.figshare.com) (you need to make use of `requests` library).

2. Extract the zip file, again programmatically, similar to how we did it in class. 

>  You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the `requests` library, similar to how we [did it in class](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture).

> There are 5 files in the figshare repo. The one we want is: `data.zip`


In [2]:
# Referenced the below code from lecture notes 

article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = os.path.join(os.path.pardir, "data/rainfall/")

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

### Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.
    
2. When combining the CSV files, add an extra column called "model" that identifies the model.
    Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON
    Tip 2: Remember how we added "year" column when we combined airline CSVs. Here the regex will be to get word before an underscore ie, "/([^_]*)"

> Note: There is a file called `observed_daily_rainfall_SYD.csv` in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from the folder) before you combine CSVs. We will use this file in our next milestone.

3. ***Compare*** run times on different machines within your team and summarize your observations. 

> Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop. 


In [4]:
%%time
files = glob.glob(os.path.join(os.path.pardir, "data/rainfall/*.csv"))
files.remove(os.path.join(os.path.pardir, "data/rainfall", "observed_daily_rainfall_SYD.csv"))

df = pd.concat((
  pd.read_csv(file, index_col=0)
  .assign(model=re.findall("([^_]*)", os.path.basename(file))[0])
  for file in files))
df.to_csv(os.path.join(os.path.pardir, "data/combined_data.csv"))

CPU times: total: 8min 56s
Wall time: 9min 27s


| Team Member | Operating System | RAM          | Processor             | Is SSD | CPU time   | Wall time  |
|:-----------:|:----------------:|:------------:|:---------------------:|:------:|:----------:|:----------:|
| Rev         |MacOS             |8GB 3733MHz   |Intel i3 1.1GHz        |Yes     |6m56s       |7m47s       |
| Caroline    |Windows 10        |16GB 3200MHz  |Intel i7-11800H 2.3GHz |Yes     |5m47s       |5m49s       |
| Sneha       |Windows 11        |16GB 4800MHz  |Intel i7-12700H 2.3GHz |Yes     |5m20s       |5m39s       |
| Renzo       |Windows 10        |8GB 2400MHz   |Intel i5-7300HQ 2.5GHz |Yes     |8m56s       |9m27s       |

All four of us were able to combine the data CSV into a single CSV using our respective systems without any errors. As shown in the table above, the runtimes varies for each of us because of different system specifications. Three of us are using windows systems and one is using Mac. The runtime is lesser in the systems having more RAM and better processors. The runtimes are comparable for all except for Renzo where the system takes few additional minutes since the system is not brand new.

### Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts). 
    - Changing `dtype` of your data
    - Load just columns that we want
    - Loading in chunks
    
2. ***Compare*** run times on different machines within your team and summarize your observations.


Saving memory usage: Using only certain columns (time, model, rain) and cast rain to float32

In [5]:
%%time
use_cols = ['time', 'rain (mm/day)', 'model']
df = pd.read_csv(os.path.join(os.path.pardir, "data/combined_data.csv"), 
    usecols = use_cols)
df['rain (mm/day)'] = df['rain (mm/day)'].astype('float32', errors='ignore')


CPU times: total: 1min 33s
Wall time: 1min 58s


In [6]:
%%time 

df['model'].value_counts()

CPU times: total: 4.02 s
Wall time: 5.19 s


MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

| Team Member | Operating System | RAM          | Processor             | Is SSD | CPU time (dtype,column)  | Wall time(dtype,column)  |
|:-----------:|:----------------:|:------------:|:---------------------:|:------:|:----------:|:----------:|
| Rev         |MacOS             |8GB 3733MHz   |Intel i3 1.1GHz        |Yes     |55.3s, 2.94s       |1m39s 3.1s       |
| Caroline    |Windows 10        |16GB 3200MHz  |Intel i7-11800H 2.3GHz |Yes     |40.1s, 2.17s       |40.8s, 2.18s      |
| Sneha       |Windows 11        |16GB 4800MHz  |Intel i7-12700H 2.3GHz |Yes     |33.6s, 1.84s       |34.2s, 2s |
| Renzo       |Windows 10        |8GB 2400MHz   |Intel i5-7300HQ 2.5GHz |Yes     |1min 33s, 4.02 s   |1min 58s, 5.19 s      |

### 6. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}

1. Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others.
    - [Parquet file](http://parquet.apache.org)
    - [Pandas exchange](https://rpy2.github.io/doc/latest/html/interactive.html)
    - [Arrow exchange](https://github.com/rpy2/rpy2-arrow)
2. Once you have the dataframe in R, perform a simple EDA.


We chose the Arrow exchange method because this method is primarily designed in such a way that the data exchange between programming languages is efficient and performance is high. The Arrow exchange method does not require serialization and de-serialization which makes the process of transfer of data between R and Python more efficient and faster. Arrow also ensures consistency and compatibility between systems and is memory efficient which makes it a good choice for our case.

In [9]:
# !pip install rpy2_arrow
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra
import os


R[write to console]: Error in gettext(fmt, domain = domain, trim = trim) : 
  3 arguments passed to .Internal(gettext) which requires 2



RRuntimeError: Error in gettext(fmt, domain = domain, trim = trim) : 
  3 arguments passed to .Internal(gettext) which requires 2


In [ ]:
%load_ext rpy2.ipython

In [ ]:
path = "/Users/revathypon/525/525-group-11/data/combined_data.csv"

In [ ]:
%%time
csv_data = ds.dataset(path, format="csv")
# Converting the `pyarrow dataset` to a `pyarrowtable`
table = csv_data.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 21.7 s, sys: 4.65 s, total: 26.4 s
Wall time: 24.7 s


In [ ]:
%%time

%%R -i r_table

suppressMessages(library(dplyr))
output <- r_table |> count(model)|> collect()
print(output)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 3.59 s, sys: 2.45 s, total: 6.04 s
Wall time: 5.36 s
